In [104]:
import re
import string
import csv
from collections import Counter
from sklearn.model_selection import train_test_split
from pyvi import ViTokenizer, ViPosTagger, ViUtils
from CocCocTokenizer import PyTokenizer
T = PyTokenizer(load_nontone_data=True)
# print(T.word_tokenize("tot  san pham giong hinhda dep", tokenize_option=0))
# print(ViTokenizer.tokenize("xin chào, tôi là người Việt Nam"))


In [1]:
stop_word = []
with open('stopword.txt') as fp:
    for row in fp:
        stop_word.append(row.strip())
stop_word = set(stop_word)

In [168]:
def replaceCompositUnicode(str):
    str = str.replace('à', 'à')
    str = str.replace('ả', 'ả')
    str = str.replace('ã', 'ã')
    str = str.replace('á', 'á')
    str = str.replace('ạ', 'ạ')
    str = str.replace('ấ', 'ấ')
    str = str.replace('ầ', 'ầ')
    str = str.replace('ậ', 'ậ')
    str = str.replace('ẩ', 'ẩ')
    str = str.replace('ẫ', 'ẫ')
    str = str.replace('ắ', 'ắ')
    str = str.replace('ằ', 'ằ')
    str = str.replace('ặ', 'ặ')
    str = str.replace('ẳ', 'ẳ')
    str = str.replace('ẵ', 'ẵ')
    str = str.replace('è', 'è')
    str = str.replace('ẻ', 'ẻ')
    str = str.replace('ẽ', 'ẽ')
    str = str.replace('é', 'é')
    str = str.replace('ẹ', 'ẹ')
    str = str.replace('ế', 'ế')
    str = str.replace('ề', 'ề')
    str = str.replace('ệ', 'ệ')
    str = str.replace('ể', 'ể')
    str = str.replace('ễ', 'ễ')
    str = str.replace('ì', 'ì')
    str = str.replace('ỉ', 'ỉ')
    str = str.replace('ĩ', 'ĩ')
    str = str.replace('í', 'í')
    str = str.replace('ị', 'ị')
    str = str.replace('ò', 'ò')
    str = str.replace('ỏ', 'ỏ')
    str = str.replace('õ', 'õ')
    str = str.replace('ó', 'ó')
    str = str.replace('ọ', 'ọ')
    str = str.replace('ố', 'ố')
    str = str.replace('ồ', 'ồ')
    str = str.replace('ộ', 'ộ')
    str = str.replace('ổ', 'ổ')
    str = str.replace('ỗ', 'ỗ')
    str = str.replace('ớ', 'ớ')
    str = str.replace('ờ', 'ờ')
    str = str.replace('ợ', 'ợ')
    str = str.replace('ở', 'ở')
    str = str.replace('ỡ', 'ỡ')
    str = str.replace('ù', 'ù')
    str = str.replace('ủ', 'ủ')
    str = str.replace('ũ', 'ũ')
    str = str.replace('ú', 'ú')
    str = str.replace('ụ', 'ụ')
    str = str.replace('ứ', 'ứ')
    str = str.replace('ừ', 'ừ')
    str = str.replace('ự', 'ự')
    str = str.replace('ử', 'ử')
    str = str.replace('ữ', 'ữ')
    str = str.replace('ỳ', 'ỳ')
    str = str.replace('ỷ', 'ỷ')
    str = str.replace('ỹ', 'ỹ')
    str = str.replace('ý', 'ý')
    str = str.replace('ỵ', 'ỵ')
    return str

def removeSpecialCharacter(str):
    # Loai bo ki tu dac biet
    str = re.sub(r'[^0-9a-zăâđêôơưàảãáạấầậẩẫắằặẳẵèẻẽéẹếềệểễìỉĩíịòỏõóọốồộổỗớờợởỡùủũúụứừựửữỳỷỹýỵ,\s\*]+', ' ',str)
    # Xoa bo khoang trang dai
    str = re.sub(' {2,}', ' ', str)
    return str

def replaceAcronym(str):
    # mot so tu viet tat
    # 1*->1 sao, 2*-> 2 sao,...
    str = re.sub('(^| )([0-9])\*($| )', r'\1sao\2', str)
    # 1sao -> 1 sao, 2sao -> 2 sao, ...
    str = re.sub('(^| )([0-9])sao($| )', r'\1sao\2', str)
    # * -> sao
    str = re.sub("(^| )\*($| )", r"\1sao\2", str)
    # s -> sao
    str = re.sub("(^| )s($| )", r"\1sao\2", str)
    str = re.sub("(^| )sp($| )", r"\1sản phẩm\2", str)
    str = re.sub("(^| )rat($| )", r"\1rất\2", str)
    str = re.sub("(^| )dang($| )", r"\1đáng\2", str)
    str = re.sub("(^| )nhiu($| )", r"\1nhiều\2", str)
    str = re.sub("(^| )nhìu($| )", r"\1nhiều\2", str)
    str = re.sub("(^| )nhieu($| )", r"\1nhiều\2", str)
    str = re.sub("(^| )tot($| )", r"\1tốt\2", str)
    str = re.sub("(^| )tôt($| )", r"\1tốt\2", str)
    str = re.sub("(^| )muot($| )", r"\1mượt\2", str)
    str = re.sub("(^| )muọt($| )", r"\1mượt\2", str)
    str = re.sub("(^| )mươt($| )", r"\1mượt\2", str)
    str = re.sub("(^| )may($| )", r"\1máy\2", str)
    str = re.sub("(^| )tr($| )", r"\1trước\2", str)
    str = re.sub("(^| )trc($| )", r"\1trước\2", str)
    str = re.sub("(^| )ae($| )", r"\1anh em\2", str)
    str = re.sub("(^| )đc($| )", r"\1được\2", str)
    str = re.sub("(^| )dc($| )", r"\1được\2", str)
    str = re.sub("(^| )dk($| )", r"\1được\2", str)
    str = re.sub("(^| )đk($| )", r"\1được\2", str)
    str = re.sub("(^| )k($| )", r"\1không\2", str)
    str = re.sub("(^| )ko($| )", r"\1không\2", str)
    str = re.sub("(^| )kg($| )", r"\1không\2", str)
    str = re.sub("(^| )kh($| )", r"\1không\2", str)
    str = re.sub("(^| )hk($| )", r"\1không\2", str)
    str = re.sub("(^| )khg($| )", r"\1không\2", str)
    str = re.sub("(^| )r($| )", r"\1rồi\2", str)
    str = re.sub("(^| )m($| )", r"\1mình\2", str)
    str = re.sub("(^| )mh($| )", r"\1mình\2", str)
    str = re.sub("(^| )mih($| )", r"\1mình\2", str)
    str = re.sub("(^| )mìh($| )", r"\1mình\2", str)
    str = re.sub("(^| )mk($| )", r"\1mình\2", str)
    str = re.sub("(^| )mik($| )", r"\1mình\2", str)
    str = re.sub("(^| )bh($| )", r"\1bao giờ\2", str)
    str = re.sub("(^| )h($| )", r"\1giờ\2", str)
    str = re.sub("(^| )jo($| )", r"\1giờ\2", str)
    str = re.sub("(^| )z($| )", r"\1vậy\2", str)
    str = re.sub("(^| )j($| )", r"\1gì\2", str)
    str = re.sub("(^| )cx($| )", r"\1cũng\2", str)
    str = re.sub("(^| )vs($| )", r"\1với\2", str)
    str = re.sub("(^| )ah($| )", r"\1à\2", str)
    str = re.sub("(^| )ak($| )", r"\1à\2", str)
    str = re.sub("(^| )ntn($| )", r"\1như thế nào\2", str)
    str = re.sub("(^| )lm($| )", r"\1làm\2", str)
    str = re.sub("(^| )trc($| )", r"\1trước\2", str)
    str = re.sub("(^| )t2($| )", r"\1thứ 2\2", str)
    str = re.sub("(^| )cn($| )", r"\1chủ nhật\2", str)
    str = re.sub("(^| )t3($| )", r"\1thứ 3\2", str)
    str = re.sub("(^| )t4($| )", r"\1thứ 4\2", str)
    str = re.sub("(^| )t5($| )", r"\1thứ 5\2", str)
    str = re.sub("(^| )t6($| )", r"\1thứ 6\2", str)
    str = re.sub("(^| )t7($| )", r"\1thứ 7\2", str)
    str = re.sub("(^| )mn($| )", r"\1mọi người\2", str)
    str = re.sub("(^| )good($| )", r"\1rất_tốt\2", str)
    str = re.sub("(^| )5 sao($| )", r"\1năm_sao\2", str)
    str = re.sub("(^| )năm sao($| )", r"\1năm_sao\2", str)
    str = re.sub("(^| )4 sao($| )", r"\1bốn_sao\2", str)
    str = re.sub("(^| )bốn sao($| )", r"\1bốn_sao\2", str)
    str = re.sub("(^| )3 sao($| )", r"\1ba_sao\2", str)
    str = re.sub("(^| )ba sao($| )", r"\1ba_sao\2", str)
    str = re.sub("(^| )2 sao($| )", r"\1hai_sao\2", str)
    str = re.sub("(^| )hai sao($| )", r"\1hai_sao\2", str)
    str = re.sub("(^| )1 sao($| )", r"\1một_sao\2", str)
    str = re.sub("(^| )một sao($| )", r"\1một_sao\2", str)
    str = re.sub("(^| )ok($| )", r"\1rất_tốt\2", str)
    str = re.sub("(^| )e($| )", r"\1em\2", str)
    str = re.sub("(^| )t($| )", r"\1tôi\2", str)
    str = re.sub("(^| )thikkk($| )", r"\1thích\2", str)
    str = re.sub("(^| )thik($| )", r"\1thích\2", str)
    str = re.sub("(^| )cx($| )", r"\1cũng\2", str)
    str = re.sub("(^| )deep($| )", r"\1đẹp\2", str)
    str = re.sub("(^| )đepj($| )", r"\1đẹp\2", str)
    return str


In [404]:
from unidecode import unidecode

def preprocess(str):
    # to lower case
    str = str.lower()
    # remove punctuation
    str = str.translate(str.maketrans("","", string.punctuation))
    # remove white spaces
    str = str.strip()
    str = re.sub(' +', ' ', str)
    str = replaceCompositUnicode(str)
    str = removeSpecialCharacter(str)
    str = replaceAcronym(str)
#     # to remove number
#     str = re.sub(r'\d+','', str)
    # tokenize by pyvi
    str = ViTokenizer.tokenize(str)
    # tokenize by coccoctokenizer
#     print(T.word_tokenize(str, tokenize_option=0))
#     str = unidecode(str)
    
    # tokenize by nltk
#     tknzr = TweetTokenizer()
#     str = tknzr.tokenize(str)
#     print(str)
    # # remove stopword
    str = str.split()
    str = [i for i in str if not i in stop_word]
    str = ' '.join(str)
#     enhance feature
#     j = 0
#     n_s = ""
#     k = int(len(str)/3 - 1) * 3
#     for i in range(3,k, 3):
#         n_s += str[j:i] + " "
#         j = i
#     n_s += str[k:]
    
    return str
    

In [405]:
def balance_classes(xs, ys):
    freqs = Counter(ys)
    max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_ys = []
    new_xs = []
    for i, y in enumerate(ys):
        if num_added[y] < max_allowable:
            new_ys.append(y)
            new_xs.append(xs[i])
            num_added[y] += 1
    return new_xs, new_ys

In [406]:
star= []
review = []
# with open('data_final2.csv') as csv_file:
#     for row in csv_file:
#         row = row.split('<fff>')
#         star.append(row[0])
#         review.append(preprocess(row[1]))

with open('train.txt', 'r') as fp:
    line = fp.readline()
    while line:
        star.append(line[0])
        review.append(preprocess(line[2:]))
        line = fp.readline()

# with open('bag_text.txt', 'r') as csv_file:
#     i = 0
#     for row in csv_file:
#         row = row.split('<fff>')
#         star.append(row[0])
#         review.append(preprocess(row[1]))


#         i += 1
# #         print(i)
#         if(i == 3000):
# #             break
# for i in range(1, 5):
#     filename = "{}_star.txt".format(i)
#     with open(filename, 'r') as fp:
#         line = fp.readline()
#         j = 0
#         while j < 500:
#             star.append('{}'.format(i))
#             review.append(preprocess(line))
#             line = fp.readline() 
#             j += 1



In [407]:
# review, star = balance_classes(review, star)
X_train, X_test, y_train, y_test = train_test_split(review, star, test_size=0.33, random_state=42)

In [408]:
Counter(y_train)

Counter({'5': 185, '4': 85, '3': 41, '2': 9, '1': 15})

In [409]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

#Train and evaluate the model
# vect = CountVectorizer().fit(X_train)
vect = TfidfVectorizer(ngram_range=(1,3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(X_train_vectorized, y_train)
preds = clfrNB.predict(vect.transform(X_test))
score = accuracy_score(y_test, preds)
print(score)
print(confusion_matrix(y_test, preds))

0.6606060606060606
[[ 4  0  1  2  9]
 [ 0  1  0  2  2]
 [ 0  0  5  4 11]
 [ 0  0  0 16 18]
 [ 0  0  0  7 83]]


In [410]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve


vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectors = vectorizer.fit_transform(X_train)
vectors1 = vectorizer.transform(X_test)
    #     return vectors

    # def fit(self):
        # vectors = self.tfidf()
classifier = LinearSVC()
        # train the classifier
classifier.fit(vectors, y_train)
        # return classifier
# vectors1 = vectorizer.transform(x_test)
predict = classifier.predict(vectors1)
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test, predict)
print(ac)
print(confusion_matrix(y_test, predict))

0.6909090909090909
[[ 7  0  2  1  6]
 [ 0  1  0  1  3]
 [ 0  0  9  1 10]
 [ 0  0  0 18 16]
 [ 0  0  1 10 79]]


In [411]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix


nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(confusion_matrix(y_test, y_pred))

accuracy 0.6060606060606061
[[ 0  0  0  0 16]
 [ 0  0  0  1  4]
 [ 0  0  3  0 17]
 [ 0  0  0 11 23]
 [ 0  0  0  4 86]]


In [412]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(confusion_matrix(y_test, y_pred))

accuracy 0.703030303030303
[[ 7  0  2  3  4]
 [ 0  2  0  1  2]
 [ 0  0 10  2  8]
 [ 0  0  1 23 10]
 [ 0  0  2 14 74]]
